In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import random
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.layers import Conv1D, MaxPooling1D, Bidirectional, LSTM, Dropout, Dense, Flatten, Attention, Layer, Concatenate, Permute, Reshape, Multiply
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
import warnings
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from keras.optimizers import Adam
from pmdarima import auto_arima
import keras.backend as K
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.0f}'.format)
import os
import tensorflow as tf
# from lssvm import LSSVR
from datetime import datetime

In [ ]:
def results(a,b,c,d,e,f,g):
    current_time = datetime.now()
    data = {
        'model' : [g],
        'sim' : [a],
        'mae' : [b],
        'rmse' : [c],
        'fsd' : [d],
        'R' : [e],
        'NSE': [f],
        'time' : [current_time]
    }
    df = pd.DataFrame(data)
    with open('results_combine.csv', 'a', newline='') as f:
        if os.path.isfile('results_combine.csv'):
            df.to_csv('results_combine.csv', mode='a', header=False, index=False)
        else:
            df.to_csv('results_combine.csv', index=False)

def to_df(data_list):
    X_df = [i[:-1] for i in data_list]
    y_df = [i[-1] for i in data_list]
    transposed_lists = [list(x) for x in zip(*X_df)]

    df_list = pd.DataFrame({f'Column{i+1}': lst for i, lst in enumerate(transposed_lists)})
    df_list['Target'] = y_df
    return df_list

def transform_to_multivariate(data, T):
    M = []
    for i in range(len(data) - T):
        row = data[i:i + T + 1]
        M.append(row)
    return np.array(M)

def calculate_similarity(value_lst_after, value_lst_before):
        T = len(value_lst_after)  # Number of missing values
        similarity_sum = 0

        for i in range(T):
            yi = value_lst_after[i]
            xi = value_lst_before[i]
            similarity_sum += 1 / (1 + abs(yi - xi) / (max(value_lst_before) - min(value_lst_before)))

        similarity = similarity_sum / T
        return similarity

def calculate_MAE(value_lst_missing, value_lst_after):
        return mean_absolute_error(value_lst_missing, value_lst_after)

def calculate_RMSE(value_lst_missing, value_lst_after):
    return np.sqrt(mean_squared_error(value_lst_missing, value_lst_after))

def calculate_FB(value_lst_missing, value_lst_after):
    return 2 * abs((np.mean(value_lst_after) - np.mean(value_lst_missing)) / (np.mean(value_lst_after) + np.mean(value_lst_missing)))

def calculate_fsd(value_lst_missing, value_lst_after):
    std_dev_Y = np.std(value_lst_after)
    std_dev_X = np.std(value_lst_missing)

    if std_dev_X == 0:
        return None
    
    fsd = 2 * abs((std_dev_Y - std_dev_X) / (std_dev_X + std_dev_Y))
    
    return fsd

def calculate_r_score(value_lst_missing, value_lst_after):

    correlation_matrix = np.corrcoef(value_lst_missing, value_lst_after)
    r_score = correlation_matrix[0, 1]
    return r_score

def calculate_nse(value_lst_missing, value_lst_after):

    value_lst_missing = np.array(value_lst_missing)
    value_lst_after = np.array(value_lst_after)

    numerator = np.sum((value_lst_missing - value_lst_after)**2)
    denominator = np.sum((value_lst_missing - np.mean(value_lst_missing))**2)

    nse = 1 - (numerator / denominator)
    
    return nse


def calculate_metrics_for_combine(value_lst_after,name_model):
    
    df_before_missing = pd.read_csv('waterlevel.csv')
    value_lst_missing = df_before_missing['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap]


    similarity_score = calculate_similarity(value_lst_after, value_lst_missing)
    MAE_score = calculate_MAE(value_lst_missing, value_lst_after)
    RMSE_score = calculate_RMSE(value_lst_missing, value_lst_after)
    FSD_score = calculate_fsd(value_lst_missing, value_lst_after)
    R_score = calculate_r_score(value_lst_missing, value_lst_after)
    NSE_score = calculate_nse(value_lst_missing, value_lst_after)
    
    sim_lst_combine.append(similarity_score)
    mae_lst_combine.append(MAE_score)
    rmse_lst_combine.append(RMSE_score)
    fsd_lst_combine.append(FSD_score)
    r_lst_combine.append(R_score)
    nse_lst_combine.append(NSE_score)

    
    print('\nOri_data:', value_lst_missing)
    print('\nvalue_data:', value_lst_after)
    print('\nSimilarity_score:', similarity_score)
    print('\nMean Absolute Error (MAE):', MAE_score)
    print('\nRoot Mean Squared Error (RMSE):', RMSE_score)
    print('\nFraction of Standard Deviation Score:', FSD_score)
    print('\nR score:', R_score)
    print('\nThe Nash Sutcliffe efficiency (NSE):', NSE_score)

    results(similarity_score, MAE_score, RMSE_score, FSD_score, R_score, NSE_score,name_model)
    
def calculate_metrics_for_LSSVM(value_lst_after,name_model):
    
    df_before_missing = pd.read_csv('waterlevel.csv')
    value_lst_missing = df_before_missing['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap]


    similarity_score = calculate_similarity(value_lst_after, value_lst_missing)
    MAE_score = calculate_MAE(value_lst_missing, value_lst_after)
    RMSE_score = calculate_RMSE(value_lst_missing, value_lst_after)
    FSD_score = calculate_fsd(value_lst_missing, value_lst_after)
    R_score = calculate_r_score(value_lst_missing, value_lst_after)
    NSE_score = calculate_nse(value_lst_missing, value_lst_after)
    
    sim_lst_LSSVM.append(similarity_score)
    mae_lst_LSSVM.append(MAE_score)
    rmse_lst_LSSVM.append(RMSE_score)
    fsd_lst_LSSVM.append(FSD_score)
    r_lst_LSSVM.append(R_score)
    nse_lst_LSSVM.append(NSE_score)

    
    print('\nOri_data:', value_lst_missing)
    print('\nvalue_data:', value_lst_after)
    print('\nSimilarity_score:', similarity_score)
    print('\nMean Absolute Error (MAE):', MAE_score)
    print('\nRoot Mean Squared Error (RMSE):', RMSE_score)
    print('\nFraction of Standard Deviation Score:', FSD_score)
    print('\nR score:', R_score)
    print('\nThe Nash Sutcliffe efficiency (NSE):', NSE_score)

    results(similarity_score, MAE_score, RMSE_score, FSD_score, R_score, NSE_score,name_model)
    
def calculate_metrics_for_Bi_LSTM(value_lst_after,name_model):
    
    df_before_missing = pd.read_csv('waterlevel.csv')
    value_lst_missing = df_before_missing['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap]


    similarity_score = calculate_similarity(value_lst_after, value_lst_missing)
    MAE_score = calculate_MAE(value_lst_missing, value_lst_after)
    RMSE_score = calculate_RMSE(value_lst_missing, value_lst_after)
    FSD_score = calculate_fsd(value_lst_missing, value_lst_after)
    R_score = calculate_r_score(value_lst_missing, value_lst_after)
    NSE_score = calculate_nse(value_lst_missing, value_lst_after)
    
    sim_lst_Bi_LSTM.append(similarity_score)
    mae_lst_Bi_LSTM.append(MAE_score)
    rmse_lst_Bi_LSTM.append(RMSE_score)
    fsd_lst_Bi_LSTM.append(FSD_score)
    r_lst_Bi_LSTM.append(R_score)
    nse_lst_Bi_LSTM.append(NSE_score)

    
    print('\nOri_data:', value_lst_missing)
    print('\nvalue_data:', value_lst_after)
    print('\nSimilarity_score:', similarity_score)
    print('\nMean Absolute Error (MAE):', MAE_score)
    print('\nRoot Mean Squared Error (RMSE):', RMSE_score)
    print('\nFraction of Standard Deviation Score:', FSD_score)
    print('\nR score:', R_score)
    print('\nThe Nash Sutcliffe efficiency (NSE):', NSE_score)

    results(similarity_score, MAE_score, RMSE_score, FSD_score, R_score, NSE_score,name_model)
    
def calculate_metrics_for_CNN(value_lst_after,name_model):
    
    df_before_missing = pd.read_csv('waterlevel.csv')
    value_lst_missing = df_before_missing['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap]


    similarity_score = calculate_similarity(value_lst_after, value_lst_missing)
    MAE_score = calculate_MAE(value_lst_missing, value_lst_after)
    RMSE_score = calculate_RMSE(value_lst_missing, value_lst_after)
    FSD_score = calculate_fsd(value_lst_missing, value_lst_after)
    R_score = calculate_r_score(value_lst_missing, value_lst_after)
    NSE_score = calculate_nse(value_lst_missing, value_lst_after)
    
    sim_lst_CNN.append(similarity_score)
    mae_lst_CNN.append(MAE_score)
    rmse_lst_CNN.append(RMSE_score)
    fsd_lst_CNN.append(FSD_score)
    r_lst_CNN.append(R_score)
    nse_lst_CNN.append(NSE_score)

    
    print('\nOri_data:', value_lst_missing)
    print('\nvalue_data:', value_lst_after)
    print('\nSimilarity_score:', similarity_score)
    print('\nMean Absolute Error (MAE):', MAE_score)
    print('\nRoot Mean Squared Error (RMSE):', RMSE_score)
    print('\nFraction of Standard Deviation Score:', FSD_score)
    print('\nR score:', R_score)
    print('\nThe Nash Sutcliffe efficiency (NSE):', NSE_score)

    results(similarity_score, MAE_score, RMSE_score, FSD_score, R_score, NSE_score,name_model)


def create_continuous_missing_values(dataframe, column_name, num_missing_values):
    modified_df = dataframe.copy()
    
    if len(dataframe) > num_missing_values:
        random_index = random.randint(0, len(dataframe) - num_missing_values)
        modified_df.loc[random_index:random_index + num_missing_values - 1, column_name] = np.nan
    else:
        print("Error: The number of missing values requested exceeds the DataFrame's capacity.")
    return modified_df


sim_lst_combine = []
mae_lst_combine = []
rmse_lst_combine = []
fsd_lst_combine = []
r_lst_combine = []
nse_lst_combine = []

sim_lst_CNN = []
mae_lst_CNN = []
rmse_lst_CNN = []
fsd_lst_CNN = []
r_lst_CNN = []
nse_lst_CNN = []

sim_lst_Bi_LSTM = []
mae_lst_Bi_LSTM = []
rmse_lst_Bi_LSTM = []
fsd_lst_Bi_LSTM = []
r_lst_Bi_LSTM = []
nse_lst_Bi_LSTM = []

sim_lst_LSSVM = []
mae_lst_LSSVM = []
rmse_lst_LSSVM = []
fsd_lst_LSSVM = []
r_lst_LSSVM = []
nse_lst_LSSVM = []

scaler = StandardScaler()

In [ ]:
# original_df_path = 'waterlevel.csv'
# try:
#     original_df = pd.read_csv(original_df_path)
    
#     for i in range(5,6):
#         modified_df = create_continuous_missing_values(original_df, 'Waterlevel', 48)
#         modified_df.to_csv(f'waterlevel_missing_{i}.csv', index=False)
        
#         print(f'waterlevel_missing_{i}.csv saved with continuous missing values.')

# except FileNotFoundError:
#     print(f"Failed to find '{original_df_path}'. Please check the file path and try again.")

In [ ]:
def model_combine(X_train):
    combine = tf.keras.models.Sequential()
    combine.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    combine.add(MaxPooling1D(pool_size=2))
    combine.add(Dropout(0.15))
    
    combine.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    combine.add(Dropout(0.15))
    
    combine.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
    combine.add(Dropout(0.15))
    
    combine.add(Bidirectional(LSTM(units=200, return_sequences=True)))
    combine.add(Dropout(0.15))
    
    class Attention(Layer):
        def __init__(self, **kwargs):
            super(Attention, self).__init__(**kwargs)
    
        def build(self, input_shape):
            self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1), 
                                     initializer='random_normal', trainable=True)
            self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1), 
                                     initializer='zeros', trainable=True)        
            super(Attention, self).build(input_shape)
     
        def call(self, x):
            e = K.tanh(K.dot(x, self.W) + self.b)
            e = K.squeeze(e, axis=-1)
            alpha = K.softmax(e)
            alpha = K.expand_dims(alpha, axis=-1)
            context = x * alpha
            context = K.sum(context, axis=1)
            return context

    combine.add(Attention())
    
    combine.add(Dense(units=200, activation='relu'))
    
    combine.add(Dense(units=1))
    
    combine.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])
    
    return combine

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.debugging.set_log_device_placement(True)

early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

for i in range(0, 1):
    
    filename = f'waterlevel_missing_{i}.csv'
    df = pd.read_csv(filename)
    
    #### Check size of missing value
    size_of_gap = df['Waterlevel'].isna().sum()

    data = df['Waterlevel'].values.tolist()

    nan_index = None
    for i, value in enumerate(data):
        if value != value:  # Check if the value is NaN
            nan_index = i
            break
        
    df_check_before = data[:(3*size_of_gap)+1]
    df_check_after = data[::-1][:3*size_of_gap+1]
    df_miss = data[nan_index:nan_index + size_of_gap]

    last_data = data[:nan_index]
    first_data = data[nan_index+size_of_gap:][::-1]

    # check if missing values is in the first 3 x T data original
    if all(value in df_check_before for value in df_miss):
        print('\nAll values in df_miss is in the first !!!')
        
        # Calculate for combine 
        first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
        df_list = to_df(first_value_combine)
        

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])

        # model, callbacks = model_combine(X)        
        # model.fit(X, y, epochs=2000, batch_size = 64, callbacks=callbacks)
        
        model = model_combine(X)
        model.fit(X, y, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)

        data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
        data_test = np.concatenate(data_test).ravel()   

        results_first_combine = []
        for i in range(len(data_test)//2):
            data_first = data_test[i:i+1+size_of_gap]

            data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

            results_first_combine.append(data_first[size_of_gap])
            
        ###################################################################
        print('\n', 'result of combine only (first):')                        #  
        calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
        print('\n')                                                       #
        ###################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_first_combine, label='Predicted Value', linestyle='--')
        plt.title('Water Level vs Predicted Value Over Time')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
    elif all(value in df_check_after for value in df_miss):
        print('\nAll values in df_miss is in the last !!!')
        
        last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
        df_list = to_df(last_value_combine)

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])

        # model, callbacks = model_combine(X)
        # model.fit(X, y, epochs=2000, batch_size = 64, callbacks=callbacks)
        
        model = model_combine(X)
        model.fit(X, y, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)

        data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test = np.concatenate(data_test).ravel()   

        results_last_combine = []
        for i in range(len(data_test)//2):
            data_last = data_test[i:i+1+size_of_gap]

            # data_last[size_of_gap] = float(model.predict(np.array(data_last[:size_of_gap]).reshape(1,-1)).ravel()[0])
            data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

            results_last_combine.append(data_last[size_of_gap])
            
        #################################################################
        print('\n', 'result of combine only (last):')                       #
        calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
        print('\n')                                                     #
        # #################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_last_combine, label='Predicted Value', linestyle='--')
        plt.title('Water Level vs Predicted Value Over Time')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
    else:
        Da = data[nan_index + size_of_gap:][::-1]

        MDa = transform_to_multivariate(Da, size_of_gap)

        df_MDa = to_df(MDa)

        X_MDa_train = np.array(df_MDa.iloc[:, :-1])
        X_MDa_train = scaler.fit_transform(X_MDa_train)
        # X_MDa_train = X_MDa_train.reshape((X_MDa_train.shape[0], X_MDa_train.shape[1], 1))
        
        y_MDa_train = np.array(df_MDa.iloc[:, -1])

        # model_MDa, callbacks_MDa = model_combine(X_MDa_train)
        # model_MDa.fit(X_MDa_train, y_MDa_train, epochs=2000, batch_size = 64, callbacks=callbacks_MDa)
        
        model_MDa = model_combine(X_MDa_train)
        model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)
        
        data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
        data_test_after = np.concatenate(data_test_after).ravel()   

        value_lst_after = []
        for j in range(len(data_test_after)//2):
            data_after = data_test_after[j:j+1+size_of_gap]
            
            data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
            value_lst_after.append(data_after[size_of_gap])

        Db = data[:nan_index]

        MDb = transform_to_multivariate(Db, size_of_gap)  

        df_MDb = to_df(MDb)

        X_MDb_train = np.array(df_MDb.iloc[:, :-1])
        X_MDb_train = scaler.fit_transform(X_MDb_train)
        # X_MDb_train = X_MDb_train.reshape((X_MDb_train.shape[0], X_MDb_train.shape[1], 1))
        
        y_MDb_train = np.array(df_MDb.iloc[:, -1])

        # model_MDb, callbacks_MDb = model_combine(X_MDb_train)
        # model_MDb.fit(X_MDb_train, y_MDb_train, epochs=2000, batch_size = 64, callbacks=callbacks_MDb)
        
        model_MDb = model_combine(X_MDb_train)
        model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)

        data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test_before = np.concatenate(data_test_before).ravel()   

        value_lst_before = []
        for i in range(len(data_test_before)//2):
            data_before = data_test_before[i:i+1+size_of_gap]
            
            data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

            value_lst_before.append(data_before[size_of_gap])

        #############################################################################
        print('\n', 'result of combine only: ')                                         #
        results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
        calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
        print('\n')                                                                 #
        #############################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_combine, label='Predicted Value', linestyle='--')
        plt.title('Water Level vs Predicted Value Over Time')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
      
current_time = datetime.now()


print('\n')
print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
print('\nMean of R-score combine: ', np.mean(r_lst_combine))
print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
print('\n')

In [ ]:
def model_Bi_LSTM(X_train):
    bi_lstm = tf.keras.models.Sequential()


    bi_lstm.add(Bidirectional(LSTM(units=200, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    bi_lstm.add(Dropout(0.15))
    
    bi_lstm.add(LSTM(units=200, return_sequences=True))
    bi_lstm.add(Dropout(0.15))
    bi_lstm.add(LSTM(units=200, return_sequences=True))
    bi_lstm.add(Dropout(0.15))
    bi_lstm.add(LSTM(units=200))
    bi_lstm.add(Dropout(0.15))
    
    bi_lstm.add(Dense(units=200, activation = "relu"))
    bi_lstm.add(Dense(units=1))
    
    
    bi_lstm.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])
    
    return bi_lstm

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.debugging.set_log_device_placement(True)

for i in range(0, 12):
    
    filename = f'waterlevel_missing_{i}.csv'
    df = pd.read_csv(filename)
    
    #### Check size of missing value
    size_of_gap = df['Waterlevel'].isna().sum()

    data = df['Waterlevel'].values.tolist()

    nan_index = None
    for i, value in enumerate(data):
        if value != value:  # Check if the value is NaN
            nan_index = i
            break
        
    df_check_before = data[:(3*size_of_gap)+1]
    df_check_after = data[::-1][:3*size_of_gap+1]
    df_miss = data[nan_index:nan_index + size_of_gap]

    last_data = data[:nan_index]
    first_data = data[nan_index+size_of_gap:][::-1]

    # check if missing values is in the first 3 x T data original
    if all(value in df_check_before for value in df_miss):
        print('\nAll values in df_miss is in the first !!!')
        
        # Calculate for Bi_LSTM 
        first_value_Bi_LSTM = transform_to_multivariate(first_data, size_of_gap)
                
        df_list = to_df(first_value_Bi_LSTM)

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])

        # model, callbacks = model_Bi_LSTM(X)        
        # model.fit(X, y, epochs=500, batch_size = 64, callbacks=callbacks)
        
        model = model_Bi_LSTM(X)
        model.fit(X, y, epochs=50, batch_size=64)

        data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
        data_test = np.concatenate(data_test).ravel()   

        results_first_Bi_LSTM = []
        for i in range(len(data_test)//2):
            data_first = data_test[i:i+1+size_of_gap]

            data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

            results_first_Bi_LSTM.append(data_first[size_of_gap])
            
        ###################################################################
        print('\n', 'result of Bi_LSTM only (first):')                        #  
        calculate_metrics_for_Bi_LSTM(results_first_Bi_LSTM,'results_Bi_LSTM (FIRST)')#
        print('\n')                                                       #
        ###################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_first_Bi_LSTM, label='Predicted Value', linestyle='--')
        plt.title('Water Level vs Predicted Value Over Time')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

        
    elif all(value in df_check_after for value in df_miss):
        print('\nAll values in df_miss is in the last !!!')
        
        last_value_Bi_LSTM = transform_to_multivariate(last_data, size_of_gap)
                
        df_list = to_df(last_value_Bi_LSTM)

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])

        # model, callbacks = model_Bi_LSTM(X)
        # model.fit(X, y, epochs=500, batch_size = 64, callbacks=callbacks)
        
        model = model_Bi_LSTM(X)
        model.fit(X, y, epochs=50, batch_size=64)

        data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test = np.concatenate(data_test).ravel()   

        results_last_Bi_LSTM = []
        for i in range(len(data_test)//2):
            data_last = data_test[i:i+1+size_of_gap]

            # data_last[size_of_gap] = float(model.predict(np.array(data_last[:size_of_gap]).reshape(1,-1)).ravel()[0])
            data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

            results_last_Bi_LSTM.append(data_last[size_of_gap])
            
        #################################################################
        print('\n', 'result of Bi_LSTM only (last):')                       #
        calculate_metrics_for_Bi_LSTM(results_last_Bi_LSTM,'results_Bi_LSTM (LAST)')#
        print('\n')                                                     #
        # #################################################################
        
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_last_Bi_LSTM, label='Predicted Value', linestyle='--')
        plt.title('Water Level vs Predicted Value Over Time')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
    else:
        Da = data[nan_index + size_of_gap:][::-1]

        MDa = transform_to_multivariate(Da, size_of_gap)

        df_MDa = to_df(MDa)

        X_MDa_train = np.array(df_MDa.iloc[:, :-1])
        X_MDa_train = scaler.fit_transform(X_MDa_train)
        # X_MDa_train = X_MDa_train.reshape((X_MDa_train.shape[0], X_MDa_train.shape[1], 1))
        
        y_MDa_train = np.array(df_MDa.iloc[:, -1])

        # model_MDa, callbacks_MDa = model_Bi_LSTM(X_MDa_train)
        # model_MDa.fit(X_MDa_train, y_MDa_train, epochs=500, batch_size = 64, callbacks=callbacks_MDa)
        
        model_MDa = model_Bi_LSTM(X_MDa_train)
        model_MDa.fit(X_MDa_train, y_MDa_train, epochs=50, batch_size=64)
        
        data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
        data_test_after = np.concatenate(data_test_after).ravel()   

        value_lst_after = []
        for j in range(len(data_test_after)//2):
            data_after = data_test_after[j:j+1+size_of_gap]
            
            data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
            value_lst_after.append(data_after[size_of_gap])

        Db = data[:nan_index]

        MDb = transform_to_multivariate(Db, size_of_gap)  

        df_MDb = to_df(MDb)

        X_MDb_train = np.array(df_MDb.iloc[:, :-1])
        X_MDb_train = scaler.fit_transform(X_MDb_train)
        # X_MDb_train = X_MDb_train.reshape((X_MDb_train.shape[0], X_MDb_train.shape[1], 1))
        
        y_MDb_train = np.array(df_MDb.iloc[:, -1])

        # model_MDb, callbacks_MDb = model_Bi_LSTM(X_MDb_train)
        # model_MDb.fit(X_MDb_train, y_MDb_train, epochs=500, batch_size = 64, callbacks=callbacks_MDb)
        
        model_MDb = model_Bi_LSTM(X_MDb_train)
        model_MDb.fit(X_MDb_train, y_MDb_train, epochs=50, batch_size=64)

        data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test_before = np.concatenate(data_test_before).ravel()   

        value_lst_before = []
        for i in range(len(data_test_before)//2):
            data_before = data_test_before[i:i+1+size_of_gap]
            
            data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

            value_lst_before.append(data_before[size_of_gap])

        #############################################################################
        print('\n', 'result of Bi_LSTM only: ')                                         #
        results_Bi_LSTM =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
        calculate_metrics_for_Bi_LSTM(results_Bi_LSTM,'results_Bi_LSTM (MIDDLE)')               #
        print('\n')                                                                 #
        #############################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_Bi_LSTM, label='Predicted Value', linestyle='--')
        plt.title('Water Level vs Predicted Value Over Time')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
current_time = datetime.now()


print('\n')
print('\nMean of Similarity Bi_LSTM: ',np.mean(sim_lst_Bi_LSTM))
print('\nMean of Mean Absoulute Error Bi_LSTM :',np.mean(mae_lst_Bi_LSTM))
print('\nMean of Root Mean Squared Error Bi_LSTM: ',np.mean(rmse_lst_Bi_LSTM))
print('\nMean of Fraction of Standard Deviation Bi_LSTM: ',np.mean(fsd_lst_Bi_LSTM)) 
print('\nMean of R-score Bi_LSTM: ', np.mean(r_lst_Bi_LSTM))
print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_Bi_LSTM))
print('\n')

In [ ]:
def model_CNN(X_train):
    
    CNN = tf.keras.models.Sequential()
    CNN.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    CNN.add(MaxPooling1D(pool_size=2))
    CNN.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    CNN.add(Dropout(0.15))
    CNN.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
    CNN.add(Dropout(0.15))
    CNN.add(Flatten())
    CNN.add(Dense(50, activation='relu'))
    CNN.add(Dropout(0.2))
    
    CNN.add(Dense(units=1))
    
    CNN.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])
    
    return CNN


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.debugging.set_log_device_placement(True)

for i in range(0,12):
    
    filename = f'waterlevel_missing_{i}.csv'
    df = pd.read_csv(filename)
    
    #### Check size of missing value
    size_of_gap = df['Waterlevel'].isna().sum()

    data = df['Waterlevel'].values.tolist()

    nan_index = None
    for i, value in enumerate(data):
        if value != value:  # Check if the value is NaN
            nan_index = i
            break
        
    df_check_before = data[:(3*size_of_gap)+1]
    df_check_after = data[::-1][:3*size_of_gap+1]
    df_miss = data[nan_index:nan_index + size_of_gap]

    last_data = data[:nan_index]
    first_data = data[nan_index+size_of_gap:][::-1]

    # check if missing values is in the first 3 x T data original
    if all(value in df_check_before for value in df_miss):
        print('\nAll values in df_miss is in the first !!!')
        
        # Calculate for CNN 
        first_value_CNN = transform_to_multivariate(first_data, size_of_gap)
                
        df_list = to_df(first_value_CNN)

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])

        # model, callbacks = model_CNN(X)        
        # model.fit(X, y, epochs=500, batch_size = 64, callbacks=callbacks)
        
        model = model_CNN(X)
        model.fit(X, y, epochs=50, batch_size=16)

        data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
        data_test = np.concatenate(data_test).ravel()   

        results_first_CNN = []
        for i in range(len(data_test)//2):
            data_first = data_test[i:i+1+size_of_gap]

            data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

            results_first_CNN.append(data_first[size_of_gap])
            
        ###################################################################
        print('\n', 'result of CNN only (first):')                        #  
        calculate_metrics_for_CNN(results_first_CNN,'results_CNN (FIRST)')#
        print('\n')                                                       #
        ###################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_first_CNN, label='Predicted Value', linestyle='--')
        plt.title('Water Level vs Predicted Value Over Time')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
    elif all(value in df_check_after for value in df_miss):
        print('\nAll values in df_miss is in the last !!!')
        
        last_value_CNN = transform_to_multivariate(last_data, size_of_gap)
                
        df_list = to_df(last_value_CNN)

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])

        # model, callbacks = model_CNN(X)
        # model.fit(X, y, epochs=500, batch_size = 64, callbacks=callbacks)
        
        model = model_CNN(X)
        model.fit(X, y, epochs=50, batch_size=16)

        data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test = np.concatenate(data_test).ravel()   

        results_last_CNN = []
        for i in range(len(data_test)//2):
            data_last = data_test[i:i+1+size_of_gap]

            # data_last[size_of_gap] = float(model.predict(np.array(data_last[:size_of_gap]).reshape(1,-1)).ravel()[0])
            data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

            results_last_CNN.append(data_last[size_of_gap])
            
        #################################################################
        print('\n', 'result of CNN only (last):')                       #
        calculate_metrics_for_CNN(results_last_CNN,'results_CNN (LAST)')#
        print('\n')                                                     #
        # #################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_last_CNN, label='Predicted Value', linestyle='--')
        plt.title('Water Level vs Predicted Value Over Time')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
    else:
        Da = data[nan_index + size_of_gap:][::-1]

        MDa = transform_to_multivariate(Da, size_of_gap)

        df_MDa = to_df(MDa)

        X_MDa_train = np.array(df_MDa.iloc[:, :-1])
        X_MDa_train = scaler.fit_transform(X_MDa_train)
        # X_MDa_train = X_MDa_train.reshape((X_MDa_train.shape[0], X_MDa_train.shape[1], 1))
        
        y_MDa_train = np.array(df_MDa.iloc[:, -1])

        # model_MDa, callbacks_MDa = model_CNN(X_MDa_train)
        # model_MDa.fit(X_MDa_train, y_MDa_train, epochs=500, batch_size = 64, callbacks=callbacks_MDa)
        
        model_MDa = model_CNN(X_MDa_train)
        model_MDa.fit(X_MDa_train, y_MDa_train, epochs=50, batch_size=16)
        
        data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
        data_test_after = np.concatenate(data_test_after).ravel()   

        value_lst_after = []
        for j in range(len(data_test_after)//2):
            data_after = data_test_after[j:j+1+size_of_gap]
            
            data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
            value_lst_after.append(data_after[size_of_gap])

        Db = data[:nan_index]

        MDb = transform_to_multivariate(Db, size_of_gap)  

        df_MDb = to_df(MDb)

        X_MDb_train = np.array(df_MDb.iloc[:, :-1])
        X_MDb_train = scaler.fit_transform(X_MDb_train)
        # X_MDb_train = X_MDb_train.reshape((X_MDb_train.shape[0], X_MDb_train.shape[1], 1))
        
        y_MDb_train = np.array(df_MDb.iloc[:, -1])

        # model_MDb, callbacks_MDb = model_CNN(X_MDb_train)
        # model_MDb.fit(X_MDb_train, y_MDb_train, epochs=500, batch_size = 64, callbacks=callbacks_MDb)
        
        model_MDb = model_CNN(X_MDb_train)
        model_MDb.fit(X_MDb_train, y_MDb_train, epochs=50, batch_size=16)

        data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test_before = np.concatenate(data_test_before).ravel()   

        value_lst_before = []
        for i in range(len(data_test_before)//2):
            data_before = data_test_before[i:i+1+size_of_gap]
            
            data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

            value_lst_before.append(data_before[size_of_gap])

        #############################################################################
        print('\n', 'result of CNN only: ')                                         #
        results_CNN =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
        calculate_metrics_for_CNN(results_CNN,'results_CNN (MIDDLE)')               #
        print('\n')                                                                 #
        #############################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_CNN, label='Predicted Value', linestyle='--')
        plt.title('Water Level vs Predicted Value Over Time')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
current_time = datetime.now()


print('\n')
print('\nMean of Similarity CNN: ',np.mean(sim_lst_CNN))
print('\nMean of Mean Absoulute Error CNN :',np.mean(mae_lst_CNN))
print('\nMean of Root Mean Squared Error CNN: ',np.mean(rmse_lst_CNN))
print('\nMean of Fraction of Standard Deviation CNN: ',np.mean(fsd_lst_CNN)) 
print('\nMean of R-score CNN: ', np.mean(r_lst_CNN))
print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_CNN))
print('\n')